In [1]:
from opentree import OT

In [2]:
ott_birds_subtree = OT.taxon_subtree(ott_id=81461, label_format="id")

In [3]:
ott_birds_ids = set()
for node in ott_birds_subtree.tree:
    if node.label:
        ott_birds_ids.add(node.label)
    if node.taxon:
        if node.taxon.label:
            ott_birds_ids.add(node.taxon.label)

In [4]:
len(ott_birds_ids)

27465

In [5]:
ott_taxonomy_file = "../../ott3.2/taxonomy.tsv"
ott_bird_tax_dict = {}
header = ['ottid', 'parent_ottid', 'name','rank','source','uniqname','flags']
bird_tax_file = open("birds_of_ott.tsv", "w")
head = open(ott_taxonomy_file).readline()
bird_tax_file.write(head)
for lin in open(ott_taxonomy_file):
        lii=lin.split('\t|\t')
        if "ott{}".format(lii[0]) in ott_birds_ids:
            ott_bird_tax_dict[lii[0]]=dict(zip(header, lii))
            bird_tax_file.write(lin)
        
bird_tax_file.close()
        

In [6]:
len(ott_bird_tax_dict)

27465

In [7]:
name_to_id = {}
for tax in ott_bird_tax_dict:
    name_to_id[ott_bird_tax_dict[tax]['name']] = ott_bird_tax_dict[tax]['ottid']

In [8]:
ebird_taxonomy_file = open("eBird_Taxonomy_v2019.csv")
header = ['TAXON_ORDER','CATEGORY','SPECIES_CODE','PRIMARY_COM_NAME','SCI_NAME','ORDER1','FAMILY','SPECIES_GROUP','REPORT_AS']
ebird_all_tax_dict = {}
ebird_taxonomy_file.readline()
for lin in ebird_taxonomy_file:
        lii=lin.split(',')
        ebird_all_tax_dict[lii[2]]=dict(zip(header, lii))

In [9]:
len(ebird_all_tax_dict)

16513

In [65]:
ebird_name_to_code = {}
for tax in ebird_all_tax_dict:
    ebird_name_to_code[ebird_all_tax_dict[tax]['SCI_NAME']] = tax

In [66]:
exact_taxon_matches = {}
for spp in ebird_all_tax_dict:
    if ebird_all_tax_dict[spp]['SCI_NAME'] in name_to_id:
        exact_taxon_matches[spp] = name_to_id[ebird_all_tax_dict[spp]['SCI_NAME']]

In [67]:
list(exact_taxon_matches.keys())[:10]

['ostric2',
 'ostric3',
 'grerhe1',
 'tabtin1',
 'higtin1',
 'hootin1',
 'grytin1',
 'soltin1',
 'blatin1',
 'gretin1']

In [68]:
#synonym_exact_matched = {}
#for spp in ebird_all_tax_dict:
#    if spp not in exact_taxon_matches:
#        if spp not in synonym_exact_matched:
#            ebname= ebird_all_tax_dict[spp]['SCI_NAME']
#            match = OT.tnrs_match(names=[ebname], context_name="Birds")
#            if match.response_dict.get('results'):
#                if len(match.response_dict['results'][0]['matches']) == 1:
#                    ott_id = match.response_dict['results'][0]['matches'][0]['taxon']['ott_id']
#                    synonym_exact_matched[spp] = ott_id
    

In [69]:
len(synonym_exact_matched)

909

In [70]:
spp

'bird1'

In [71]:
#synmatchedfi = open('exact_synonym_match.csv', 'w')
#synmatchedfi.write("ebird_code, ott_id\n")
#for spp in synonym_exact_matched:
#    synmatchedfi.write("{},{}\n".format(spp, synonym_exact_matched[spp]))

#synmatchedfi.close()
    

In [72]:
synmatchedfi = open('exact_synonym_match.csv')
synonym_exact_matched = {}
for lin in synmatchedfi:
    lii = lin.strip().split(',')
    synonym_exact_matched[lii[0]] = lii[1]

In [73]:
#unmatchedfi = open('unmatched.txt','w')
#for spp in ebird_all_tax_dict:
#    if spp not in exact_taxon_matches:
#        if spp not in synonym_exact_matched:
#            unmatchedfi.write(ebird_all_tax_dict[spp]['SCI_NAME'])
#            unmatchedfi.write('\n')
            
#unmatchedfi.close()

In [74]:
reverse_name = {}
for spp in ebird_all_tax_dict:
    reverse_name[ebird_all_tax_dict[spp]['SCI_NAME']] = spp

In [75]:
fuzzy_matches = {}
fuzzy_match_fi = open("output/fuzzy_matches.csv")
fuzzy_match_fi.readline()
for lin in fuzzy_match_fi:
    lii = lin.split('\t')
    ott_id = lii[2]
    if len(ott_id) > 1:
        spp = reverse_name[lii[0]]
        fuzzy_matches[spp] = ott_id

In [76]:
eliot_matches = {}
eliot_match_fi = open("final-eBird-ott7Mar2021.csv")
eliot_match_fi.readline()
new_synonmys = open("synonym_ammendments.csv", 'w')
new_synonmys.write("ott_id, current_name, new_synonym\n")
new_taxa = open("taxon_ammendments.csv", 'w')
new_taxa.write("genus,new_taxon\n")
for lin in eliot_match_fi:
    lii = lin.split(',')
    ebname = lii[0]
    synonym = lii[1].strip()
    if len(synonym.split(' ')) >= 2:
        match = OT.tnrs_match(names=[synonym], context_name="Birds")
        if match.response_dict.get('results'):
                if len(match.response_dict['results'][0]['matches']) == 1:
                    ott_id = match.response_dict['results'][0]['matches'][0]['taxon']['ott_id']
                    code = ebird_name_to_code[ebname]
                    eliot_matches[code] = ott_id
                    new_synonmys.write("{},{},{}\n".format(ott_id, synonym, ebname))
        else:
            print(synonym)
    else:
        new_taxa.write(lin)
        
new_synonmys.close()
new_taxa.close()

In [100]:
ofi = open("combined_data.tsv", "w")
header = ['TAXON_ORDER','CATEGORY','SPECIES_CODE','PRIMARY_COM_NAME','SCI_NAME','ORDER1','FAMILY','SPECIES_GROUP','REPORT_AS', 'ottid', 'parent_ottid', 'name','rank','source','uniqname','flags']
ofi.write('\t'.join(header))
ofi.write('\tmatch_type\n')
all_ott_ids = {}
for spp in ebird_all_tax_dict:
    ott_id = None
    typ = ""
    if spp in exact_taxon_matches:
        ott_id = exact_taxon_matches[spp]
        typ = "canonical_match"
    if spp in synonym_exact_matched:
        ott_id = synonym_exact_matched[spp]
        typ = "synonym_match"
    if spp in fuzzy_matches:
        ott_id = fuzzy_matches[spp]
        typ = "fuzzy_match"
    if spp in eliot_matches:
        ott_id = eliot_matches[spp]
        typ = "hand_match_eliot"
    info = ebird_all_tax_dict[spp]
    if ott_id:
        info.update(ott_bird_tax_dict[str(ott_id)])
        for item in header:
            ofi.write(info.get(item,'-').strip())
            ofi.write('\t')
        ofi.write(typ)
        all_ott_ids[str(ott_id)] = (info['SPECIES_CODE'], info['SCI_NAME'])
    else:
        for item in header:
            ofi.write(info.get(item,'-').strip())
            ofi.write('\t')
        ofi.write("unmatched")
    ofi.write('\n')
ofi.close()
            
        

        
        

In [78]:
len(all_ott_ids)

11339

In [79]:
len(all_ott_ids) - len(ebird_all_tax_dict)

-5174

In [80]:
#ofi = open("ott_taxa_not_in_ebird.csv", 'w')
#header = ['ottid', 'parent_ottid', 'name','rank','source','uniqname','flags']
#ofi.write('\t'.join(header))
#for ott_id in ott_bird_tax_dict:
#    if ott_id not in all_ott_ids:
#        for item in header:
#            ofi.write(ott_bird_tax_dict[str(ott_id)][item].strip())
#            ofi.write('\t')
#        ofi.write('\n')
        
#ofi.close()
        

In [59]:
output = OT.synth_induced_tree(ott_ids=list(all_ott_ids.keys()),  label_format='id', ignore_unknown_ids= True)
output.tree.write(path="all_ebird_id_label.tre",schema="newick")


Unknown/unrecognized query ids (skipped):
 ott119218
ott190899
ott3596323
ott3596700
ott3596720
ott3597941
ott3598459
ott3598824
ott3599194
ott3599195
ott3599401
ott3599746
ott3599748
ott3599973
ott3599997
ott369200
ott4947472
ott4947755
ott5223331
ott5557490
ott5560018
ott5560040
ott5560046
ott5560054
ott5560059
ott5560085
ott5560140
ott5560141
ott5857014
ott5857028
ott5857150
ott5857176
ott6153484
ott7068071
ott7068110
ott7068136
ott7068293
ott7068423
ott7068450
ott7068681
ott7068710
ott7068754
ott7068756
ott7068809
ott7068810
ott7068862
ott7658001
ott7659997
ott7659998
ott7659999
ott7660000
ott7660001
ott7660002
ott7660003
ott7660004
ott7660005
ott7661494
ott7662947
ott7663542
ott793528
ott86671
ott877560
ott9050
ott938420 


In [81]:
import json
# From git@github.com:OpenTreeOfLife/ChronoSynth.git
dates = json.load(open("../../ChronoSynth/node_ages.json"))

In [82]:
#Blad demands a date for the root. I picked this one form google...
dates['node_ages']['ott81461'] = {'age': 60,
  'source_id': 'google',
  'source_node': None,
  'time_unit': 'Myr'},

dates['node_ages']['ott241846'] = {'age': 60,
  'source_id': 'google',
  'source_node': None,
  'time_unit': 'Myr'},

In [83]:
trefile = "ebird_Aves_ID_annot.tre"
#Get the synthetic tree from OpenTree and write out the citations to a text file.
output.tree.write(path = trefile, schema = "newick")

In [84]:
ages = open("ages",'w')

dated_nodes = set()
undated_nodes = set()
for node in output.tree:
    lab = None
    if node.label:
        if node.label.startswith('mrca'):
            lab = node.label
        elif node.label.startswith('ott'):
            lab = node.label
        else:
            lab = node.label.split()[-1]
        if lab in dates['node_ages']:
            dated_nodes.add(lab)
            age_range = [float(source['age']) for source in dates['node_ages'][lab]]
            age_range.sort()
            age_est = sum(age_range) / len(age_range) 
            ages.write("{}\t{}\n".format(node.label, age_est))
        else:
            undated_nodes.add(lab)
    if node.taxon:
        if node.taxon.label:
            if node.taxon.label.startswith('mrca'):
                lab = node.taxon.label
            elif node.taxon.label.startswith('ott'):
                lab = node.taxon.label
            else:
                lab = node.taxon.label.split()[-1]
            if lab in dates['node_ages']:
                dated_nodes.add(lab)
                age_range = [float(source['age']) for source in dates['node_ages'][lab]]
                age_range.sort()
                age_est = sum(age_range) / len(age_range) 
                ages.write("{}\t{}\n".format(node.label, age_est))
            else:
                undated_nodes.add(lab)


len(dated_nodes)

6354

In [85]:
! phylocom bladj -f ebird_Aves_ID_annot.tre > all_ebird_blad.tre

In [86]:
ages = open("ages",'w')

dated_nodes = set()
undated_nodes = set()
for node in output.tree:
    lab = None
    if node.label:
        if node.label.startswith('mrca'):
            lab = node.label
        elif node.label.startswith('ott'):
            lab = node.label
        else:
            lab = node.label.split()[-1]
        if lab in dates['node_ages']:
            dated_nodes.add(lab)
            age_range = [float(source['age']) for source in dates['node_ages'][lab]]
            age_range.sort()
            age_est = sum(age_range) / len(age_range) 
            ages.write("{}\t{}\n".format(node.label, age_est))
        else:
            undated_nodes.add(lab)
    if node.taxon:
        if node.taxon.label:
            if node.taxon.label.startswith('mrca'):
                lab = node.taxon.label
            elif node.taxon.label.startswith('ott'):
                lab = node.taxon.label
            else:
                lab = node.taxon.label.split()[-1]
            if lab in dates['node_ages']:
                dated_nodes.add(lab)
                age_range = [float(source['age']) for source in dates['node_ages'][lab]]
                age_range.sort()
                age_est = age_range[:-1]
                ages.write("{}\t{}\n".format(node.label, age_est))
            else:
                undated_nodes.add(lab)


len(dated_nodes)

! phylocom bladj -f ebird_Aves_ID_annot.tre > all_ebird_blad_max.tre

In [87]:
ages = open("ages",'w')

dated_nodes = set()
undated_nodes = set()
for node in output.tree:
    lab = None
    if node.label:
        if node.label.startswith('mrca'):
            lab = node.label
        elif node.label.startswith('ott'):
            lab = node.label
        else:
            lab = node.label.split()[-1]
        if lab in dates['node_ages']:
            dated_nodes.add(lab)
            age_range = [float(source['age']) for source in dates['node_ages'][lab]]
            age_range.sort()
            age_est = sum(age_range) / len(age_range) 
            ages.write("{}\t{}\n".format(node.label, age_est))
        else:
            undated_nodes.add(lab)
    if node.taxon:
        if node.taxon.label:
            if node.taxon.label.startswith('mrca'):
                lab = node.taxon.label
            elif node.taxon.label.startswith('ott'):
                lab = node.taxon.label
            else:
                lab = node.taxon.label.split()[-1]
            if lab in dates['node_ages']:
                dated_nodes.add(lab)
                age_range = [float(source['age']) for source in dates['node_ages'][lab]]
                age_range.sort()
                age_est = age_range[0]
                ages.write("{}\t{}\n".format(node.label, age_est))
            else:
                undated_nodes.add(lab)


len(dated_nodes)

! phylocom bladj -f ebird_Aves_ID_annot.tre > all_ebird_blad_min.tre

In [123]:
import random
ages = open("ages",'w')

all_studies = set()
dated_nodes = set()
undated_nodes = set()
for node in output.tree:
    lab = None
    if node.label:
        if node.label.startswith('mrca'):
            lab = node.label
        elif node.label.startswith('ott'):
            lab = node.label
        else:
            lab = node.label.split()[-1]
        if lab in dates['node_ages']:
            studies = {source["source_id"].split('@')[0] for source in dates['node_ages'][lab]}
            all_studies = all_studies.union(studies)
            dated_nodes.add(lab)
            age_range = [float(source['age']) for source in dates['node_ages'][lab]]
            age_range.sort()
            age_est = sum(age_range) / len(age_range) 
            ages.write("{}\t{}\n".format(node.label, age_est))
        else:
            undated_nodes.add(lab)
    if node.taxon:
        if node.taxon.label:
            if node.taxon.label.startswith('mrca'):
                lab = node.taxon.label
            elif node.taxon.label.startswith('ott'):
                lab = node.taxon.label
            else:
                lab = node.taxon.label.split()[-1]
            if lab in dates['node_ages']:
                studies = {source["source_id"].split('@')[0] for source in dates['node_ages'][lab]}
                all_studies = all_studies.union(studies)
                dated_nodes.add(lab)
                age_range = [float(source['age']) for source in dates['node_ages'][lab]]
                age_range.sort()
                age_est = random.choice(age_range)
                ages.write("{}\t{}\n".format(node.label, age_est))
            else:
                undated_nodes.add(lab)


len(dated_nodes)

! phylocom bladj -f ebird_Aves_ID_annot.tre > all_ebird_blad_rand1.tre

{'google'}
{'google'}
{'ot_534', 'ot_809', 'pg_2913', 'ot_531', 'ot_500', 'pg_435'}
{'ot_500'}
{'ot_500'}
{'ot_500'}
{'ot_500'}
{'ot_531', 'ot_809', 'ot_500'}
{'ot_534', 'ot_500'}
{'ot_531', 'ot_809', 'ot_500'}
{'ot_531', 'ot_809', 'ot_500'}
{'ot_534', 'ot_809', 'pg_2913', 'ot_531', 'ot_500'}
{'ot_534', 'ot_809', 'ot_409', 'pg_2913', 'ot_531', 'ot_500', 'pg_435'}
{'ot_534', 'ot_809', 'ot_409', 'ot_532', 'pg_2913', 'ot_531', 'ot_500', 'ot_290', 'pg_435', 'ot_757'}
{'ot_534', 'ot_809', 'ot_409', 'ot_532', 'ot_531', 'ot_500', 'ot_290', 'ot_757'}
{'ot_809', 'ot_531', 'ot_500', 'ot_783', 'ot_290', 'ot_757'}
{'ot_809', 'ot_531', 'ot_500', 'ot_783', 'ot_290', 'ot_757'}
{'ot_531', 'ot_783', 'ot_290'}
{'ot_531', 'ot_783', 'ot_290'}
{'ot_783', 'ot_532'}
{'ot_809', 'ot_783'}
{'ot_809', 'ot_532', 'ot_531', 'ot_783', 'ot_1195', 'ot_757'}
{'ot_500', 'ot_783', 'ot_290'}
{'ot_783'}
{'ot_783'}
{'ot_783'}
{'ot_809', 'pg_2913', 'ot_290', 'ot_531', 'ot_783', 'ot_1195'}
{'ot_531', 'pg_2913', 'ot_783', 'ot_

{'ot_809', 'ot_409'}
{'ot_809', 'ot_409', 'pg_2913'}
{'ot_809', 'ot_409', 'pg_2913'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'pg_2913'}
{'pg_2913'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409', 'pg_2913'}
{'ot_809'}
{'ot_809', 'ot_409'}
{'ot_809'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809'}
{'ot_809', 'pg_2913'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'pg_2913'}
{'ot_809', 'ot_409', 'pg_2913'}
{'pg_2913'}
{'ot_809', 'ot_409'}
{'pg_2913', 'ot_783', 'ot_290'}
{'ot_1195', 'ot_783', 'ot_290'}
{'ot_1195', 

{'ot_520', 'ot_809', 'ot_409'}
{'ot_520', 'ot_809', 'ot_409'}
{'ot_520', 'ot_809', 'ot_409'}
{'ot_520', 'ot_409'}
{'ot_520'}
{'ot_520', 'ot_809', 'ot_409'}
{'ot_520', 'ot_809', 'ot_409'}
{'ot_520'}
{'ot_520', 'ot_809', 'ot_409'}
{'ot_520'}
{'ot_520'}
{'ot_520', 'ot_809', 'ot_409'}
{'ot_520', 'ot_409'}
{'ot_520', 'ot_809', 'ot_409'}
{'pg_2702'}
{'pg_2702'}
{'pg_2702'}
{'pg_2702'}
{'pg_2702'}
{'ot_520'}
{'ot_520'}
{'ot_520'}
{'ot_520', 'ot_809', 'ot_783', 'ot_290'}
{'ot_520', 'ot_809'}
{'ot_520', 'ot_409'}
{'ot_520', 'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_520', 'ot_809', 'ot_409'}
{'ot_520'}
{'ot_520'}
{'ot_520'}
{'ot_520'}
{'ot_520'}
{'ot_520'}
{'ot_520'}
{'ot_520', 'ot_809', 'ot_409'}
{'ot_520', 'ot_809', 'ot_409'}
{'ot_520'}
{'ot_520', 'ot_809', 'ot_409'}
{'ot_520', 'ot_809', 'ot_409'}
{'ot_520'}
{'ot_520', 'ot_409'}
{'ot_520', 'ot_809', 'ot_409'}
{'ot_520', 'ot_809', 'ot_409'}
{'ot_520', 'ot_809', 'ot_409'}
{'ot_520', 'ot_809', 'ot_409'}
{'ot_520', 'ot_809', 'ot_409'}
{'ot_520

{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809'}
{'ot_809', 'ot_409', 'ot_290'}
{'ot_809', 'ot_409'}
{'ot_409'}
{'ot_809'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809'}
{'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809'}
{'ot_809'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409', 'ot_290'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409', 'ot_1424'}
{'ot_809', 'ot_409', 'ot_1424'}
{'ot_809', 'ot_409'}
{'ot_809'

{'pg_2853'}
{'pg_2853'}
{'ot_809', 'ot_409', 'pg_2853'}
{'pg_2853'}
{'pg_2853'}
{'pg_2853'}
{'pg_2853'}
{'ot_809', 'ot_409', 'pg_2853'}
{'pg_2853'}
{'pg_2853'}
{'ot_809', 'ot_409', 'pg_2853'}
{'ot_809', 'pg_2853'}
{'ot_809', 'ot_409', 'pg_2853'}
{'pg_2853'}
{'pg_2853'}
{'ot_809', 'ot_409', 'pg_2853'}
{'pg_2853'}
{'ot_809', 'ot_409', 'pg_2853'}
{'ot_809', 'ot_409', 'pg_2853'}
{'ot_809', 'pg_2853'}
{'pg_2853'}
{'ot_409', 'pg_2853'}
{'ot_409'}
{'ot_809', 'ot_409', 'pg_2853'}
{'pg_2853'}
{'pg_2853'}
{'ot_409', 'pg_2853'}
{'pg_2853'}
{'pg_2853'}
{'ot_809', 'ot_409', 'pg_2853'}
{'pg_2853'}
{'ot_809', 'ot_409', 'pg_2853'}
{'ot_409', 'pg_2853'}
{'pg_2853'}
{'pg_2853'}
{'pg_2853'}
{'ot_809', 'ot_409', 'pg_2853'}
{'ot_809', 'ot_409', 'pg_2853'}
{'ot_809', 'ot_409', 'pg_2853'}
{'ot_809', 'ot_409', 'pg_2853'}
{'ot_809', 'ot_409', 'pg_2853'}
{'pg_2853'}
{'pg_2853'}
{'ot_809', 'ot_409', 'pg_2853'}
{'ot_809', 'ot_409', 'pg_2853'}
{'ot_809', 'ot_409', 'pg_2853'}
{'ot_809', 'ot_409', 'pg_2853'}
{'ot_80

{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_409'}
{'ot_809', 'ot_409'}
{'ot_809', 'ot_409'}
{'ot_534', 'ot_809', 'ot_409', 'pg_2913', 'ot_531', 'ot_500', 'pg_435'}
{'ot_1044', 'ot_534', 'pg_2926', 'ot_531', 'ot_500'}
{'ot_1044', 'ot_534', 'pg_2926', 'ot_531', 'ot_500'}
{'ot_1044', 'ot_534', 'ot_809', 'ot_409', 'pg_2926', 'ot_531'}
{'ot_1044', 'pg_2926'}
{'ot_409'}
{'ot_1044', 'ot_809', 'ot_409', 'pg_2926'}
{'ot_1044', 'ot_534', 'ot_809', 'ot_409', 'pg_2926', 'ot_531', 'ot_500', 'pg_435'}
{'ot_1044', 'pg_2926'}
{'ot_1044', 'ot_534', 'ot_809', 'ot_409', 'pg_2926', 'ot_500', 'pg_435'}
{'ot_809'}
{'ot_809', 'ot_409'}
{'ot_409'}
{'ot_809', 'ot_500'}
{'ot_809'}
{'ot_409'}
{'ot_409'}
{'ot_409'}
{'ot_409'}
{'ot_409'}
{'ot_409'}
{'ot_809'}
{'ot_409'}
{'ot_809', 'ot_409'}


In [124]:
len(all_studies)

52

In [89]:
import dendropy
bl_tree = dendropy.Tree.get_from_path("all_ebird_blad.tre", schema = "newick")

In [105]:
id_to_name = {}

In [107]:
for node in bl_tree:
    ebird_name = None
    if node.label:
        if node.label.startswith('ott'):
            print(node.label)
            ebird_name = all_ott_ids.get(node.label.strip('ott'), [None, None])[1]
            if ebird_name:
                print(ebird_name)
                id_to_name[node.label] = ebird_name
    if node.taxon:
        if node.taxon.label:
            if node.taxon.label.startswith('ott'):
                print(node.taxon.label)
                ebird_name = all_ott_ids.get(node.taxon.label.strip('ott'), [None, None])[1]
                if ebird_name:
                    print(ebird_name)
                    id_to_name[node.taxon.label] = ebird_name

ott81461
ott241846
ott1041547
ott443268
ott246
Polioptila guianensis
ott3599111
Polioptila schistaceigula
ott3599115
Polioptila clementsi
ott3599116
Polioptila paraensis
ott3599112
Polioptila lactea
ott3599113
Polioptila lembeyei
ott443265
Polioptila albiloris
ott443272
Polioptila californica
ott443273
Polioptila melanura
ott1019726
Polioptila nigriceps
ott227269
Polioptila plumbea
ott7068208
Polioptila plumbea atricapilla
ott7068209
Polioptila plumbea innotata
ott7068210
Polioptila plumbea parvirostris
ott7068211
Polioptila plumbea plumbea
ott443266
Polioptila caerulea
ott7068191
Polioptila caerulea caerulea
ott7068192
Polioptila caerulea cozumelae
ott885158
Polioptila dumicola
ott3599114
Polioptila facilis
ott5341372
Polioptila attenboroughi
ott6152054
Polioptila plumbea maior
ott7660032
Polioptila albiventris
ott713368
ott976082
Microbates cinereiventris
ott1088732
Microbates collaris
ott227270
ott227268
Ramphocaenus melanurus
ott7068218
Ramphocaenus sticturus
ott744782
ott18313
Thr

ott3597170
Psarocolius cassini
ott909234
Psarocolius viridis
ott939934
Psarocolius angustifrons
ott939938
Psarocolius atrovirens
ott699301
Psarocolius wagleri
ott125175
Cassiculus melanicterus
ott6153462
Cacicus latirostris
ott858285
ott858282
Amblycercus holosericeus
ott699304
Amblycercus holosericeus australis
ott489145
ott489144
Sturnella magna
ott283666
Sturnella neglecta
ott979985
Leistes militaris
ott858295
ott858292
Dolichonyx oryzivorus
ott364336
ott364331
Xanthocephalus xanthocephalus
ott808916
ott449562
Teretistris fernandinae
ott3597217
Teretistris fornsi
ott449581
ott449580
Zeledonia coronata
ott998132
ott998133
Icteria virens
ott303085
ott254662
Spindalis dominicensis
ott303111
Spindalis zena
ott3597339
Spindalis portoricensis
ott3597340
Spindalis nigricephala
ott884087
ott884090
Nesospingus speculiferus
ott757867
ott309808
Microligea palustris
ott808918
ott808917
Xenoligea montana
ott943069
ott943092
Phaenicophilus palmarum
ott951093
Phaenicophilus poliocephalus
ott440797

ott3597872
Eurocephalus ruppelli
ott3597878
ott3597871
Corvinella melanoleuca
ott817406
ott72471
ott20996
Hypothymis helenae
ott131232
Hypothymis coelestis
ott131264
Hypothymis puella
ott593814
Hypothymis azurea
ott537964
ott846276
Terpsiphone atrocaudata
ott72480
Terpsiphone paradisi
ott780370
Terpsiphone cyanescens
ott767075
Terpsiphone viridis
ott950262
Terpsiphone rufocinerea
ott374078
Terpsiphone batesi
ott950259
Terpsiphone bedfordi
ott585662
Terpsiphone rufiventer smithii
ott675818
Terpsiphone rufiventer
ott283818
Terpsiphone atrochalybeia
ott616161
Terpsiphone mutata
ott255598
Terpsiphone corvina
ott780374
Terpsiphone bourbonnensis
ott1011708
Terpsiphone cinnamomea
ott780372
Terpsiphone cinnamomea unirufa
ott6152418
Terpsiphone affinis
ott6152425
Terpsiphone incei
ott20990
ott542034
Trochocercus nitens
ott542038
Trochocercus cyanomelas
ott355988
Monarcha castaneiventris
ott23803
Monarcha castaneiventris ugiensis
ott850061
Monarcha castaneiventris megarhynchus
ott921355
Monarcha

ott541986
ott541985
Polioxolmis rufipennis
ott20893
ott105302
Myiotheretes striaticollis
ott541982
Myiotheretes fumigatus
ott4947640
Myiotheretes pernix
ott4947641
Myiotheretes fuscorufus
ott612077
ott305334
Agriornis micropterus
ott727889
Agriornis murinus
ott1036002
Agriornis montanus
ott3599206
Agriornis albicauda
ott3599207
Agriornis lividus
ott541980
ott541979
Neoxolmis rufiventris
ott599071
Xolmis coronatus
ott599072
Xolmis rubetra
ott599074
Xolmis cinereus
ott1036000
Xolmis pyrope
ott454007
Xolmis dominicanus
ott541981
Xolmis irupero
ott886242
Xolmis velatus
ott459135
Muscisaxicola albilora
ott567823
Muscisaxicola alpinus
ott1086405
Muscisaxicola capistratus
ott1086417
Muscisaxicola frontalis
ott1086412
Muscisaxicola maclovianus
ott1086399
Muscisaxicola rufivertex
ott509706
Muscisaxicola rufivertex pallidiceps
ott509707
Muscisaxicola rufivertex occipitalis
ott1086408
Muscisaxicola cinereus
ott1086403
Muscisaxicola flavinucha
ott567830
Muscisaxicola albifrons
ott1086410
Muscisaxi

ott3596385
Todiramphus lazuli
ott3596386
Todiramphus nigrocyaneus
ott3596388
Todiramphus australasia
ott3596389
Todiramphus gambieri
ott3596390
Todiramphus recurvirostris
ott3596392
Todiramphus enigma
ott5857392
Todiramphus gertrudae
ott5857394
Todiramphus winchelli
ott3596378
Todiramphus saurophagus
ott5859974
Todiramphus saurophagus saurophagus
ott5859984
Todiramphus albicilla
ott5859986
Todiramphus sordidus
ott3596381
Todiramphus veneratus
ott5859998
Todiramphus veneratus veneratus
ott5859999
Todiramphus veneratus youngi
ott178964
Todiramphus cinnamominus
ott5860006
Todiramphus cinnamominus cinnamominus
ott6151352
Todiramphus colonus
ott6151358
Todiramphus pelewensis
ott6151359
Todiramphus reichenbachii
ott6151360
Todiramphus sacer
ott6151363
Todiramphus tristrami
ott944058
ott571605
Actenoides lindsayi
ott571609
Actenoides concretus
ott3596406
Actenoides princeps
ott3596407
Actenoides hombroni
ott3596409
Actenoides monachus
ott6151305
Actenoides monachus capucinus
ott3596408
Acteno

Coua cursor
ott3596328
Coua coquereli
ott787064
Coua reynaudii
ott787066
Coua ruficeps
ott787072
Coua caerulea
ott3596330
Coua verreauxi
ott787074
Coua cristata
ott1040900
ott1039514
ott212737
Centropus phasianinus
ott4947394
Centropus bernsteini
ott4947387
Centropus violaceus
ott212748
Centropus viridis
ott4947399
Centropus bengalensis
ott478750
Centropus grillii
ott4947390
Centropus toulou
ott4947393
Centropus goliath
ott1039519
Centropus sinensis
ott4947398
Centropus andamanensis
ott4947396
Centropus nigrorufus
ott455567
Centropus monachus
ott595243
Centropus superciliosus
ott1016598
Centropus cupreicaudus
ott734886
Centropus senegalensis
ott4947386
Centropus leucogaster
ott4947391
Centropus anselli
ott4947383
Centropus celebensis
ott4947389
Centropus steerii
ott4947397
Centropus rectunguis
ott4947395
Centropus melanops
ott4947400
Centropus chlororhynchos
ott595245
Centropus unirufus
ott4947384
Centropus chalybeus
ott4947392
Centropus menbeki
ott4947388
Centropus ateralbus
ott494740

ott1073494
ott599077
Tinamotis ingoufi
ott599078
Tinamotis pentlandii
ott3600812
ott3600813
Nothocercus nigrocapillus
ott3600814
Nothocercus julius
ott3600815
Nothocercus bonapartei
ott829553
ott745449
ott745450
ott857860
Rhea pennata
ott857863
ott857864
Rhea americana
ott857849
ott292466
ott647692
Struthio camelus
ott3600823
Struthio molybdophanes


In [112]:
tips = [tip.taxon.label for tip in bl_tree.leaf_node_iter()]
nodes = [node for node in bl_tree]

In [113]:
len(tips)

10879

In [114]:
len(nodes)

20184

In [116]:
len(nodes)-len(tips)

9305

In [125]:
for tip in tips:
    if tip.startswith('ott'):
        assert tip in id_to_name

In [126]:
node_annotation = {}
for node in bl_tree:
    if node.label:
        node_annotation[node.label] = {}
    elif node.taxon:
        if node.taxon.label:
            node_annotation[node.taxon.label] = {}
    else:
        print(node)

In [127]:
for nid in node_annotation:
    node_annotation[nid] = {}
    node_annotation[nid]['studies'] = []
    node_annotation[nid]['strict_support'] = []
    node_annotation[nid]['conflict'] = []

In [128]:
nid_annotation = {}
for node in output.tree:
    if node.label:
        nid_annotation[node.label] = {}
    elif node.taxon:
        if node.taxon.label:
            nid_annotation[node.taxon.label] = {}
    else:
        print(node)

nid_resp = dict()
resp = OT.synth_node_info(list(nid_annotation.keys())).response_dict
for node_info in resp:
    nid_resp[node_info['node_id']] = node_info
    nid_resp[node_info['node_id']]['dates'] =  len(dates['node_ages'].get(node_info['node_id'],[]))
    


In [129]:
supported_tips = set()
unsupported_tips = set()
i = 0
for node in node_annotation:
    i+=1
    supporting = nid_resp[node].get('source_id_map')
    strict_support = nid_resp[node].get('supported_by')
    conflict = nid_resp[node].get('conflicts_with', [])
    node_annotation[node]['dates'] = nid_resp[node]['dates']
    if supporting.keys() == set(['ott3.2draft9']):
        node_annotation[node]['studies'] = 0
    else:
        node_annotation[node]['studies'] = len(supporting.keys())
    if strict_support.keys() == set(['ott3.2draft9']):
        node_annotation[node]['strict_support'] = 0
    else:
        node_annotation[node]['strict_support'] = len(strict_support.keys())
    node_annotation[node]['conflict'] = len(conflict)


In [130]:
max_conf=0
max_support=0
max_dates =0
for node in node_annotation:
    if node_annotation[node]['conflict'] > max_conf:
        max_conf = node_annotation[node]['conflict']
    if node_annotation[node]['studies'] > max_support:
        max_support = node_annotation[node]['studies']
    if node_annotation[node]['dates'] > max_dates:
        max_dates = node_annotation[node]['dates']

        

print(max_conf)
print(max_support)
print(max_dates)


17
25
29


In [131]:
fi = open("support_anno.txt", 'w')
startstr = """DATASET_STYLE
SEPARATOR TAB

#label is used in the legend table (can be changed later)
DATASET_LABEL\t{}

#dataset color (can be changed later)
COLOR\t#ffff00

DATA\n""".format("Support")
fi.write(startstr)
for node in node_annotation:
    if node_annotation[node]['studies']:
        nodelab = node.replace(" ","_")
        relsupport = node_annotation[node]['studies']/5
        r = 0
        g = 255*relsupport
        b = 0
        color = "rgba({}, {}, {}, {})".format(r, g, b, 0.25+relsupport)
        fi.write("{}\tbranch\tclade\t{}\t1\tnormal\n".format(nodelab,color))
    else:
        color = "rgba(0, 0, 0, 0.25)"
        fi.write("{}\tbranch\tclade\t{}\t1\tnormal\n".format(nodelab,color))
for tip in tips:
    color = "rgba(0, 0, 0, 0.25)"
    fi.write("{}\tbranch\tclade\t{}\t1\tnormal\n".format(tip,color))
    
fi.close()


In [132]:
fi = open("conflict_anno.txt", 'w')
startstr = """DATASET_STYLE
SEPARATOR TAB

#label is used in the legend table (can be changed later)
DATASET_LABEL\t{}

#dataset color (can be changed later)
COLOR\t#ffff00

DATA\n""".format("conflict")
fi.write(startstr)
for node in node_annotation:
    nodelab = node.replace(" ","_")
    relconf = min(node_annotation[node]['conflict'],5)/5
    r = 255*relconf
    g = 0
    b = 0
    color = "rgba({}, {}, {}, {})".format(r, g, b, 0.25+relconf)
    fi.write("{}\tbranch\tclade\t{}\t1\tnormal\n".format(nodelab,color))
for tip in tips:
    color = "rgba(0, 0, 0, 0.25)"
    fi.write("{}\tbranch\tclade\t{}\t1\tnormal\n".format(tip,color))     
fi.close()

In [133]:
fi = open("date_anno.txt", 'w')
startstr = """DATASET_STYLE
SEPARATOR TAB

#label is used in the legend table (can be changed later)
DATASET_LABEL\t{}

#dataset color (can be changed later)
COLOR\t#ffff00

DATA\n""".format("dates")
fi.write(startstr)
for node in node_annotation:
    nodelab = node.replace(" ","_")
    reldates = min(node_annotation[node]['dates'],5)/5
    r = 0
    g = 0
    b = 255*reldates
    color = "rgba({}, {}, {}, {})".format(r, g, b, 0.25+reldates)
    fi.write("{}\tbranch\tclade\t{}\t1\tnormal\n".format(nodelab,color))
        
fi.close()

In [134]:
fi = open("relabel.txt", 'w')
startstr = """LABELS
#use this template to change the leaf labels, or define/change the internal node names (displayed in mouseover popups)

#lines starting with a hash are comments and ignored during parsing

#=================================================================#
#                    MANDATORY SETTINGS                           #
#=================================================================#
#select the separator which is used to delimit the data below (TAB,SPACE or COMMA).This separator must be used throughout this file (except in the SEPARATOR line, which uses space).

#SEPARATOR TAB
#SEPARATOR SPACE
SEPARATOR COMMA

#Internal tree nodes can be specified using IDs directly, or using the 'last common ancestor' method described in iTOL help pages
#=================================================================#
#       Actual data follows after the "DATA" keyword              #
#=================================================================#
DATA
#NODE_ID,LABEL\n"""
fi.write(startstr)

for ottid in id_to_name:
    fi.write("{},{}\n".format(ottid, id_to_name[ottid]))
fi.close()

In [135]:
'ott892276' in tips

True

In [ ]:
tax